# **Projeto IA - Connect 4**

### Membros do Grupo:
- Maximiliano Vítor Phillips e Sá (up202305979)
- Orlando Miguel Carvalho-Soares (up202303606)
- Rui Jorge Pereira Rua (up202305259)

## Índice
1. Introdução

2. Código Universal

3. Algoritmos Usados
    - Monte Carlo Tree Search (MCTS)
    - Árvores de Decisão (Procedimento ID3)
        - Script para Gerar o Dataset da Árvore de Decisão
        - Pequeno Extrato do Dataset
4. Game Modes
    - Humano x Humano
    - Humano x Computador (MCTS)
    - Humano x Computador (Árvores de Decisão)
    - Computador x Computador (MCTS x Árvores de Decisão)
5. Problemas-Soluções

6. Conclusão

## 1. Introdução

Este projeto tem como objetivo desenvolver e implementar um programa capaz de jogar 'Connect-4' usando dois algoritmos diferentes. O programa tem 4 modos: Humano-Humano, Humano-Computador (MCTS), Humano-Computador (Árvores de Decisão) e Computador-Computador. O programa utiliza os algoritmos MCTS e Árvores de Decisão (usando o procedimento ID3) no modo Computador-Computador.


## 2. Código Universal

O seguinte código é 'universal' e serve para gerir o estado atual do jogo.

- "meta.py"" serve para guardar as definições básicas do jogo e o valor C para o algoritmo MCTS.

- "ConnectState.py" guarda o estado do jogo e as funções básicas para UI output e para verificar se existe jogadas legais e se existe uma vitória. 

meta.py

In [ ]:
import math

#meta dados relativos ao jogo
class GameMeta:
    PLAYERS = {'none': 0, 'one': 1, 'two': 2}
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    INF = float('inf')
    ROWS = 6
    COLS = 7
    GAMEMODE = None

#valor C do UCB
class MCTSMeta:
    C = math.sqrt(2)

ConnectState.py

In [ ]:
from copy import deepcopy
import numpy as np
from meta import GameMeta


class ConnectState:
    '''
        Construtor do board e de alguns metadados sobre o estado atual. 
        Ex: jogador atual, jogada anterior, altura de cada coluna (height[col] = 0 -> coluna col cheia)
    '''

    def __init__(self):
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        self.to_play = GameMeta.PLAYERS['one']
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        self.last_played = []

    #retorna uma cópia do estado atual
    def get_board(self):
        return deepcopy(self.board)
    
    #dá print_board ao board atual na consola
    def print_board(self):
        print('=============================')

        for row in range(GameMeta.ROWS):
            for col in range(GameMeta.COLS):
                print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
            print('|')

        print('=============================')
        print('  1   2   3   4   5   6   7')

    #marca a jogada escolhida pelo jogador no board, e dá update a alguns metadados do estado atual
    def move(self, col):
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

    #retorna uma lista com as jogadas que ainda não estão cheias (height[col] != 0)
    def get_legal_moves(self):
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

    #verfica se há vencedor e retorna o seu número (X = 1 ; O = 2), se não houver retorna 0
    def check_win(self):
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0

    def check_win_from(self, row, col):
        player = self.board[row][col]
        """
        Ultima jogada realizada está na posição (row, col)
        Verificar a grid 6x7 à volta à procura de um vencedor
        """

        consecutive = 1
        # verificar horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # verificar vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal1
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal2
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False

    #verifica se o jogo já acabou verificando se há vencedor ou se ainda existem jogadas possíveis
    def game_over(self):
        return self.check_win() or len(self.get_legal_moves()) == 0

    #verifica quem ganhou ou se é empate
    def get_result(self):
        winner = self.check_win()
        if len(self.get_legal_moves()) == 0 and winner == 0:
            return GameMeta.OUTCOMES['draw']
        return GameMeta.OUTCOMES['one'] if winner == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']



## 3. Algoritmos Usados

Foram usados dois algoritmos diferentes:
1. Monte Carlo Tree Search (MCTS)
2. Árvores de Decisão (Procedimento ID3)

### 3.1. Monte Carlo Tree Search (MCTS)


O ficheiro `mcts.py` contém duas classes principais que implementam o algoritmo MCTS para um jogo de Connect4:

### 1. Classe `Node`
Armazena toda a informação necessária para cada nó da árvore de decisão:
- Jogada realizada no nó atual
- Nó pai
- Número de visitas
- Número de vitórias
- Lista de nós filhos
- Resultado do nó

Inclui também:
- Função `value()` para calcular o UCB (Upper Confidence Bound)
- Função para adicionar nós filhos

### 2. Classe `MCTS`
Gerencia toda a lógica da busca MCTS, com:

#### Funcionalidades principais:
1. **Busca inteligente** (`search(time_limit)`):
   - Verifica vitória instantânea (`check_instant_win()`)
   - Bloqueia jogadas do adversário (`check_block_opponent()`)
   - Identifica jogadas únicas disponíveis (`check_one_move_available()`)
   
   Se nenhuma dessas situações ocorrer, executa:
   - Seleção do nó (`select_node()`)
   - Expansão (`expand()`)
   - Simulação (`simulate()`)
   - Retropropagação (`back_propagate()`)

2. **Melhor jogada** (`best_move()`):
   - Considera apenas jogadas "seguras" (que não permitem vitória imediata do adversário)
   - Escolhe a jogada com maior valor UCB entre as seguras

3. **Ajuste dinâmico** (`change_c_value()`):
   - Altera o parâmetro de exploração C de √2 (foco em exploração) para 1 (maior foco no ataque)

#### Por que é inteligente?
O MCTS tradicional tem limitações - ele nem sempre consegue simular o futuro próximo com as mesma eficácia de um humano. A nossa implementação adiciona camadas de inteligência:
- Verificações especiais para situações críticas
- Filtro de jogadas seguras
- Ajuste dinâmico do parâmetro de exploração

Isso garante que o algoritmo seja eficiente e tome decisões estratégicas, não apenas estatísticas.

mcts.py

In [ ]:
import random
import time
import math
from copy import deepcopy

from ConnectState import ConnectState
from meta import GameMeta, MCTSMeta


class Node:
    '''
        construtor da classe Node que guarda valores como o numero de visitas ao Nó, 
        número de vitórias, os seus filhos, o seu pai
    '''
    def __init__(self, move, parent):
        self.move = move
        self.parent = parent
        self.N = 0 # num times visited (current node)
        self.W = 0 # num wins for current node
        self.children = {}
        self.outcome = GameMeta.PLAYERS['none']

    #adiciona os filhos ao dicionário do "children" do pai
    def add_children(self, children: dict) -> None:
        for child in children:
            self.children[child.move] = child

    #calcular UCB do nó
    def value(self, c: float = MCTSMeta.C):
        if self.N == 0:
            return 0 if c == 0 else GameMeta.INF
        else:
            return self.W / self.N + c * math.sqrt(math.log(self.parent.N) / self.N)


class MCTS:
    #Construtor da classe MCTS que guarda algumas estatísticas e é o ponto inicial do algoritmo
    def __init__(self, state=ConnectState()):
        self.root_state = deepcopy(state)
        self.root = Node(None, None)
        self.run_time = 0
        self.node_count = 0
        self.num_rollouts = 0
        self.best_trivial = None  # armazena jogada imediata/defensiva

    # verfica se o computador pode ganhar numa jogada só
    def check_instant_win(self, state: ConnectState) -> int:
        for move in state.get_legal_moves():
            st = deepcopy(state)
            st.move(move)
            if st.game_over():
                return move
        return -1
    
    def check_block_opponent(self, state: ConnectState) -> int:
        '''Se o oponente tem jogada de vitória imediata, retorna movimento para bloqueá-la.'''
        opponent = 3 - state.to_play
        for move in state.get_legal_moves():
            st = deepcopy(state)
            # força o próximo para oponente
            st.to_play = opponent
            st.move(move)
            if st.game_over():
                return move
        return -1
    
    # verfica se o computador só tem uma coluna para jogar
    def check_one_move_available(self, state: ConnectState) -> int:
        legalMoves = state.get_legal_moves()
        if len(legalMoves) == 1:
            return legalMoves[0]
        else:
            return -1

    #escolhe o próximo nó/estado a ser explorado
    def select_node(self) -> tuple:
        node = self.root
        state = deepcopy(self.root_state)

        #escolhe o nó filho com maior UCB (se o pai já tiver sido expandido)
        while len(node.children) != 0:
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            max_nodes = [n for n in children if n.value() == max_value]

            node = random.choice(max_nodes)
            state.move(node.move)

            if node.N == 0:
                return node, state
            
        #escolhe aleatóriamente um nó filho para expandir
        if self.expand(node, state):
            node = random.choice(list(node.children.values()))
            state.move(node.move)

        return node, state

    #adiciona os estados seguintes como nós filho do estado atual
    def expand(self, parent: Node, state: ConnectState) -> bool:
        if state.game_over():
            return False

        children = [Node(move, parent) for move in state.get_legal_moves()]
        parent.add_children(children)

        return True

    #simula aleatóriamente um jogo e retorna o vencedor
    def simulate(self, state: ConnectState) -> int:
        while not state.game_over():
            state.move(random.choice(state.get_legal_moves()))

        return state.get_result()

    #faz backtrack aos nós escolhidos na fase de rollout e dá update aos seus valores
    def back_propagate(self, node: Node, turn: int, outcome: int) -> None:
        # Para o jogador atual
        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.W += reward
            node = node.parent
            if outcome == GameMeta.OUTCOMES['draw']:
                reward = 0
            else:
                reward = 1 - reward

    #procura a melhor jogada por "time_limit" segundos
    def search(self, time_limit: int):
        # 1) Vitória imediata
        win = self.check_instant_win(self.root_state)
        if win != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = win
            return
        # 2) Bloquear vitória do oponente
        block = self.check_block_opponent(self.root_state)
        if block != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = block
            return
        # 3) Única jogada possível
        one = self.check_one_move_available(self.root_state)
        if one != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = one
            return
        start_time = time.process_time()

        num_rollouts = 0
        while time.process_time() - start_time < time_limit:
            node, state = self.select_node()
            outcome = self.simulate(state)
            self.back_propagate(node, state.to_play, outcome)
            num_rollouts += 1

        self.run_time = time.process_time() - start_time
        self.num_rollouts = num_rollouts
        self.best_trivial = None

    #retorna a melhor jogada que o mcts encontrou
    def best_move(self):
        # Se root já concluída por heurística:
        if self.best_trivial is not None:
            return self.best_trivial
        
        if self.root_state.game_over():
            return -1
        
        safes = []
        for move, node in self.root.children.items():
            copy_state = deepcopy(self.root_state)
            copy_state.move(move)
            # se o oponente não consegue ganhar depois da nossa jogada
            if self.check_instant_win(copy_state) == -1:
                safes.append(node)
        '''
        # test print to check safes
        print("safes: ")
        s = []
        for i in range(len(safes)):
            s.append(safes[i].move)
        print(s)
        '''
        if safes:
            best_safe = max(safes, key=lambda n: n.value())
            return best_safe.move
        else:   
            max_value = max(self.root.children.values(), key=lambda n: n.N).N
            max_nodes = [n for n in self.root.children.values() if n.N == max_value]
            best_child = random.choice(max_nodes)
            return best_child.move

    #realiza (em definitivo) a melhor jogada que encontrou
    def move(self, move):
        if move in self.root.children:
            self.root_state.move(move)
            self.root = self.root.children[move]
            return

        self.root_state.move(move)
        self.root = Node(None, None)

    #retorna estatísticas
    def statistics(self) -> tuple:
        return self.num_rollouts, self.run_time
    
    # mudar o valor de c para 'atacar' mais em vez de focar em exploração
    def change_c_value(self):
        MCTSMeta.C = 1


### 3.2. Árvores de Decisão (Procedimento ID3)

### 1. Funções Fundamentais

1. **`entropy(y)`** - Calcula a entropia de um conjunto de dados:
   - Mede a impureza dos dados (quanto maior, mais desordenado)
   - Usa a fórmula: -Σ(p * log₂(p)) para cada classe

2. **`info_gain(y, y_left, y_right)`** - Calcula o ganho de informação:
   - Compara a entropia antes e depois de uma divisão
   - Fórmula: Entropia(pai) - [P(esquerda)*Entropia(esquerda) + P(direita)*Entropia(direita)]

### 2. Classe `Node`
Representa cada nó da árvore com:
- `feature`: Índice da característica usada para dividir
- `threshold`: Valor de corte para dados numéricos
- `left`/`right`: Subárvores esquerda e direita
- `label`: Classe (somente em nós folha)

### 3. Classe Principal `ID3TreeNumeric`

#### Método `fit(X, y)`
Constrói a árvore recursivamente:
1. **Critérios para parar de crescer**:
   - Todos os exemplos são da mesma classe
   - Não há mais características para dividir

2. **Seleção da melhor divisão**:
   - Testa todos os limiares únicos de cada característica
   - Escolhe a divisão com maior ganho de informação

3. **Construção recursiva**:
   - Divide os dados e constrói subárvores

#### Métodos de Predição
- `predict_one(x, node)`: Classifica um exemplo percorrendo a árvore
- `predict(X)`: Chama o (predict_one()) para cada elemento do array

### 4. Funções Auxiliares

1. **`manual_train_test_split()`**:
   - Divide manualmente os dados em treino/teste
   - Útil quando não se quer usar scikit-learn

2. **`trainTree()`**:
   - Carrega dados de um arquivo JSON (mcts_dataset.json)
   - Prepara os dados e treina a árvore
   - Retorna o modelo treinado

### 5. Diferenças do ID3 Tradicional
- Originalmente, ID3 trabalha com dados categóricos
- Esta versão foi adaptada para:
  - Lidar com características numéricas (usando limiares)
  - Não implementa poda (simplicidade didática)
  - Usa ganho de informação em vez de índice Gini


tree.py

In [ ]:
import numpy as np
from collections import Counter
import json

def entropy(y):
    counts = Counter(y)
    probs = [c / len(y) for c in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def info_gain(y, y_left, y_right):
    p_left = len(y_left) / len(y)
    p_right = len(y_right) / len(y)
    return entropy(y) - (p_left * entropy(y_left) + p_right * entropy(y_right))

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, label=None):
        self.feature = feature
        self.threshold = threshold  # Use for numeric split
        self.left = left
        self.right = right
        self.label = label

class ID3TreeNumeric:
    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y):
        if len(set(y)) == 1:
            return Node(label=y[0])
        if X.shape[1] == 0:
            most_common = Counter(y).most_common(1)[0][0]
            return Node(label=most_common)

        best_gain = -1
        best_feature = None
        best_threshold = None
        best_splits = None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left_idx = X[:, feature] <= t
                right_idx = X[:, feature] > t
                if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
                    continue
                gain = info_gain(y, y[left_idx], y[right_idx])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = t
                    best_splits = (left_idx, right_idx)

        if best_gain == -1:
            most_common = Counter(y).most_common(1)[0][0]
            return Node(label=most_common)

        left = self._build_tree(X[best_splits[0]], y[best_splits[0]])
        right = self._build_tree(X[best_splits[1]], y[best_splits[1]])
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

    def predict_one(self, x, node):
        if node.label is not None:
            return node.label
        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        else:
            return self.predict_one(x, node.right)

    def predict(self, X):
        return np.array([self.predict_one(x, self.root) for x in X])

# custom data split function (manual train-test split)
def manual_train_test_split(X, y, test_size=0.2):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    split = int(len(X) * (1 - test_size))
    train_indices = indices[:split]
    test_indices = indices[split:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

def trainTree():
    with open('mcts_dataset.json') as f:
        data = json.load(f)

    # Convert to features (X) and labels (y)
    X = []
    y = []

    for entry in data:
        board = np.array(entry['state']).flatten()  # Flatten 6x7 into 42-length vector
        move = entry['recommended_move']
        X.append(board)
        y.append(move)

    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)
    tree = ID3TreeNumeric()
    tree.fit(X_train, y_train)
    print("Tree trained successfully.")
    return tree

'''
# Load the dataset
ds = pd.read_csv('../iris.csv')

X = ds.iloc[:, :-1].to_numpy()  # Features
y = ds.iloc[:, -1].to_numpy()   # Labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = ID3TreeNumeric()
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

# Show the first 5 rows
print(ds.head())

# Shows each collumn data type
print(ds.dtypes)

# Checks for missing values
print(ds.isnull().values.any())
'''

#### 3.2.1. Script para Gerar o Dataset da Árvore de Decisão

O código seguinte foi utilizado para gerar um conjunto de dados de pares (estado, jogada recomendada) que será utilizado pelo algoritmo da árvore de decisão. O algoritmo MCTS foi utilizado para simular automaticamente os jogos.

dataset.py

In [ ]:
import json
from mcts import MCTS
from ConnectState import ConnectState
from copy import deepcopy
import os

def board_to_list(board):
    return [row[:] for row in board]

# return player symbol
def get_player_symbol(player_num):
    return 'X' if player_num == 1 else 'O'

def generate_dataset_json(num_games, search_time, filename="mcts_dataset.json"):
    dataset = []

    # carrega o dataset existente (se houver) para continuar a adicionar pares
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            try:
                dataset = json.load(f)
            except json.JSONDecodeError:
                dataset = []

    for game_index in range(num_games):
        state = ConnectState()
        mcts = MCTS(state)
        move_count = 1
        changed_To_Attack = False # se o valor de C foi mudado ou não

        print(f"\n=== Início do Jogo {game_index + 1} ===\n")
        state.print_board()

        while not state.game_over():
            current_board = deepcopy(state.get_board())

            mcts.search(search_time)
            best_move = mcts.best_move()

            # add (state, move) pair to dataset list 
            dataset.append({
                "state": board_to_list(current_board),
                "recommended_move": best_move
            })

            # print move made and board state
            current_player = get_player_symbol(state.to_play)
            print(f"\n Jogada {move_count} | Jogador '{current_player}' joga na coluna {best_move + 1}")
            state.move(best_move)
            state.print_board()

            mcts.move(best_move)
            move_count += 1

            if not changed_To_Attack:
                if move_count > 17:
                    mcts.change_c_value()
                    changed_To_Attack = True

    # write dataset to file
    with open(filename, 'w') as f:
        json.dump(dataset, f, indent=2)

    # print current total num of pairs
    print(f"\n[✓] Dataset guardado em '{filename}' com {len(dataset)} pares totais.")

if __name__ == "__main__":
    print("Starting sim...")
    generate_dataset_json(10, 10)


#### 3.2.2. Pequeno Extrato do Dataset

Segue-se um pequeno extrato do conjunto de dados a utilizar pelo algoritmo da árvore de decisão. O formato json foi utilizado pela sua "facilidade de utilização" e por ser leve/eficiente. O formato json permite também ler facilmente os pares (estado, jogada recomendada).

mcts_dataset.json

In [ ]:
[
  {
    "state": [
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0]
    ],
    "recommended_move": 0
  },
  {
    "state": [
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [1, 0, 0, 0, 0, 0, 0]
    ],
    "recommended_move": 4
  },
  {
    "state": [
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [1, 0, 0, 0, 2, 0, 0]
    ],
    "recommended_move": 2
  },
  {
    "state": [
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 0, 2, 0, 0]
    ],
    "recommended_move": 1
  },
  {
    "state": [
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0],
      [1, 2, 1, 0, 2, 0, 0]
    ],
    "recommended_move": 2
  }
]

## 4. Game Modes

Existem 4 modos de jogo:
1. Humano x Humano
2. Humano x Computador (MCTS)
3. Humano x Computador (Árvores de Decisão)
4. Computador x Computador (MCTS x Árvores de Decisão)

### Código Base para a Interface

game.py

In [ ]:
from ConnectState import ConnectState
from mcts import MCTS
from meta import GameMeta
from tree import trainTree
import numpy as np
import random

def playPvP():
    state = ConnectState()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Player 1 Move ---
        while True:
            p1_input = input("Player 1 ('X') enter a move: ")
            if not p1_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p1Move = int(p1_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p1Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p1Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Player 2 Move ---
        while True:
            p2_input = input("Player 2 ('O') enter a move: ")
            if not p2_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p2Move = int(p2_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p2Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p2Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break


def playPvC1():
    state = ConnectState()
    mcts = MCTS(state)
    changed_To_Attack = False # se o valor de C foi mudado ou não
    num_Plays = 0

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Human Player Move ---
        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        # --- MCTS Move ---
        state.move(user_move)
        mcts.move(user_move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        print("Thinking...")

        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")
        move = mcts.best_move()
        state.move(move)
        mcts.move(move)

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True

def playPvC2():
    state = ConnectState()
    tree = trainTree()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Human Player Move ---
        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Decision Tree Move ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]
        legal_moves = state.get_legal_moves()

        if predicted_move not in legal_moves:
            state.move(legal_moves[random.randint(0, len(legal_moves)-1)])
        else:
            state.move(predicted_move)

        print("Tree chose to play in column: " + str(predicted_move+1))

        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

def playCvC():
    state = ConnectState()
    tree = trainTree()
    mcts = MCTS(state)
    changed_To_Attack = False
    num_Plays = 0
    print("Current state:")
    state.print_board()

    while not state.game_over():
        # --- MCTS turn ---
        print("Thinking (MCTS)...")
        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")

        mcts_move = mcts.best_move()
        state.move(mcts_move)
        mcts.move(mcts_move)

        print("MCTS chose column:", mcts_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("MCTS ('X') won!")
            else:
                print("Draw!")
            break

        # --- Tree turn ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()
        if predicted_move not in legal_moves:
            # fallback to random legal move
            predicted_move = random.choice(legal_moves)

        state.move(predicted_move)
        mcts.move(predicted_move)

        print("Tree chose column:", predicted_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Tree ('O') won!")
            else:
                print("Draw!")
            break

        # Optionally adjust MCTS’s C parameter after a number of full turns
        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True
    


'''
    Inicializador do modo de jogo pretendido:
    1. PvP
    2. PvC (mcts)
    3. PvC (tree)
    4. CvC
'''
if __name__ == "__main__":
    print("Welcome to Connect 4!\nPlease select one of the following game modes by selecting its corresponding number:\n1. Human vs. Human\n2. Human vs. Computer(mcts)\n3. Human vs. Computer(tree)\n4. Computer vs. Computer")
    GameMeta.GAMEMODE = int(input("Select option:"))
    while 1 > GameMeta.GAMEMODE or GameMeta.GAMEMODE > 4:
        print("Invalid Gamemode")
        GameMeta.GAMEMODE = int(input("Select option:"))
    if GameMeta.GAMEMODE == 1:
        playPvP()
    elif GameMeta.GAMEMODE == 2:
        playPvC1()
    elif GameMeta.GAMEMODE == 3:
        playPvC2()
    else:
        playCvC()


### 4.1. Humano x Humano

Este modo permite que dois jogadores humanos se enfrentem.

1. **Inicialização**  
   - Cria um novo estado de jogo  
   - Jogador 1 (X) e Jogador 2 (O).

2. **Turno do Jogador 1**  
   - Exibe o tabuleiro atual  
   - Solicita input do jogador, validando a entrada  
   - Executa o movimento escolhido  

3. **Verificação de Estado**  
   - Após cada movimento, verifica se o jogo terminou  
   - Se terminado, exibe o resultado final  

4.  **Turno do Jogador 2**  
    - Exibe o tabuleiro atual  
    - Solicita input do jogador, validando a entrada  
    - Executa o movimento escolhido 

5. **Finalização**  
   - Exibe o tabuleiro final  
   - Mostra o resultado do jogo (vitória de um jogador ou empate)  


In [ ]:
def playPvP():
    state = ConnectState()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Player 1 Move ---
        while True:
            p1_input = input("Player 1 ('X') enter a move: ")
            if not p1_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p1Move = int(p1_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p1Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p1Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Player 2 Move ---
        while True:
            p2_input = input("Player 2 ('O') enter a move: ")
            if not p2_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p2Move = int(p2_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p2Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p2Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

### 4.2. Humano x Computador (MCTS)

Este modo permite que um jogador humano enfrente uma IA baseada no MCTS.

1. **Inicialização**  
   - Cria um novo estado de jogo  
   - O jogador humano começa sempre primeiro (como Jogador 1 - X)

2. **Turno do Jogador Humano**  
   - Exibe o tabuleiro atual  
   - Solicita input do jogador, validando a entrada  
   - Executa o movimento escolhido  

3. **Verificação de Estado**  
   - Após cada movimento, verifica se o jogo terminou  
   - Se terminado, exibe o resultado final  

4.  **Turno da IA (MCTS)**
- **Busca Inteligente**: A IA simula milhares de possibilidades em 10 segundos
- **Adaptação**: 
  - *Fase Inicial*: Explora diversas estratégias (C=√2)
  - *Após 17 jogadas*: Foca nas melhores jogadas (C=1)
- **Decisão**: Escolhe a jogada mais promissora

5. **Finalização**  
   - Exibe o tabuleiro final  
   - Mostra o resultado do jogo (vitória de um jogador ou empate)  


In [ ]:
def playPvC1():
    state = ConnectState()
    mcts = MCTS(state)
    changed_To_Attack = False # se o valor de C foi mudado ou não
    num_Plays = 0

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Human Player Move ---
        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        # --- MCTS Move ---
        state.move(user_move)
        mcts.move(user_move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        print("Thinking...")

        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")
        move = mcts.best_move()
        state.move(move)
        mcts.move(move)

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True

### 4.3. Humano x Computador (Árvores de Decisão)

Este modo permite que um jogador humano enfrente uma IA baseada em árvore de decisão.

1. **Inicialização**  
   - Cria um novo estado de jogo  
   - O jogador humano começa sempre primeiro (como Jogador 1 - X)

2. **Turno do Jogador Humano**  
   - Exibe o tabuleiro atual  
   - Solicita input do jogador, validando a entrada  
   - Executa o movimento escolhido  

3. **Verificação de Estado**  
   - Após cada movimento, verifica se o jogo terminou  
   - Se terminado, exibe o resultado final  

4. **Turno da IA (Árvore de Decisão)**  
   - Transforma o estado atual em formato adequado para a árvore  
   - Obtém a previsão do melhor movimento  
   - Garante movimento válido (usando fallback aleatório se necessário)  
   - Executa o movimento da IA  

5. **Finalização**  
   - Exibe o tabuleiro final  
   - Mostra o resultado do jogo (vitória de um jogador ou empate)  


In [ ]:
def playPvC2():
    state = ConnectState()
    tree = trainTree()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Human Player Move ---
        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Decision Tree Move ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]
        legal_moves = state.get_legal_moves()

        if predicted_move not in legal_moves:
            state.move(legal_moves[random.randint(0, len(legal_moves)-1)])
        else:
            state.move(predicted_move)

        print("Tree chose to play in column: " + str(predicted_move+1))

        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

### 4.4. Computador x Computador (MCTS x Árvores de Decisão)

Este modo permite visualizar um jogo de MCTS vs Árvore.

1. **Inicialização**  
   - Cria um novo estado de jogo  
   - Inicializa o MCTS e a Árvore

2. **Turno da IA (MCTS)**
- **Busca Inteligente**: A IA simula milhares de possibilidades em 10 segundos
- **Adaptação**: 
  - *Fase Inicial*: Explora diversas estratégias (C=√2)
  - *Após 17 jogadas*: Foca nas melhores jogadas (C=1)
- **Decisão**: Escolhe a jogada mais promissora

3. **Verificação de Estado**  
   - Após cada movimento, verifica se o jogo terminou  
   - Se terminado, exibe o resultado final  

4. **Turno da IA (Árvore de Decisão)**  
   - Transforma o estado atual em formato adequado para a árvore  
   - Obtém a previsão do melhor movimento  
   - Garante movimento válido (usando fallback aleatório se necessário)  
   - Executa o movimento da IA  

5. **Finalização**  
   - Exibe o tabuleiro final  
   - Mostra o resultado do jogo (vitória de um jogador ou empate)  


In [ ]:
def playCvC():
    state = ConnectState()
    tree = trainTree()
    mcts = MCTS(state)
    changed_To_Attack = False
    num_Plays = 0
    print("Current state:")
    state.print_board()

    while not state.game_over():
        # --- MCTS turn ---
        print("Thinking (MCTS)...")
        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")

        mcts_move = mcts.best_move()
        state.move(mcts_move)
        mcts.move(mcts_move)

        print("MCTS chose column:", mcts_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("MCTS ('X') won!")
            else:
                print("Draw!")
            break

        # --- Tree turn ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()
        if predicted_move not in legal_moves:
            # fallback to random legal move
            predicted_move = random.choice(legal_moves)

        state.move(predicted_move)
        mcts.move(predicted_move)

        print("Tree chose column:", predicted_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Tree ('O') won!")
            else:
                print("Draw!")
            break

        # Optionally adjust MCTS’s C parameter after a number of full turns
        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True

## 5. Problemas - Soluções

#### -MCTS:

   1. `Problema`: O MCTS às vezes não bloqueava uma jogada que daria vitória ao adversário.

      `Solução`: Criamos a função check_block_opponent() para bloquear se ocorrer essa situação.
      <br>
      <br>

   2. `Problema`: O MCTS às vezes não jogava numa posição que lhe daria a vitória.

      `Solução`: Criamos a função check_instant_win() para jogar nessa posição e ganhar.
      <br>
      <br>
   
   3. `Problema`: O MCTS às vezes jogava numa posição que dava a vitória na jogada seguinte ao adversário.

      `Solução`: Modificamos o best_move() para verificar as jogadas seguras (jogadas que não dessem a vitória ao adversário na jogada seguinte).
      <br>
      <br>

   4. `Problema`: O MCTS jogava muito defensivamente o que possibilitava vencê-lo com uma certa facilidade no fim do jogo.

      `Solução`: Mudar o valor de C para 1 a meio do jogo para que fosse mais agressivo.

#### -Árvore de Decisão:

   1. `Problema`: Depois de treinado, o output da árvore para um  dado valor é determinístico. Por isso dando como input o estado atual do jogo, a árvore vai sempre dar a mesma jogada como output, independemente das vezes que tentarmos dar-lhe esse input. Mas existe a possibilidade do output da árvore ser uma jogada inválida (porque a coluna já está cheia).

      `Solução`: No caso de a jogada ser inválida escolhemos uma coluna aleatóriamente (dentro das colunas válidas).

## 6. Conclusão

Depois de muitos testes e muito tempo dedicado ao aperfeiçoamento das implementações, chegamos à conclusão que o MCTS é uma melhor solução para este problema.

### Razões:

####  1. Utilização de recursos

O MCTS joga em tempo real ou quase, logo não necessita de nenhum tipo de treino ou geração de dados prévios. A geração do dataset usado na árvore demorou horas e horas a gerar (e provavelmente convinha ser ainda maior), gastando eltricidade e recursos computcionais imensos.

####  2. Rendimento

A árvore tem um rendimento muito inferior ao MCTS. Apesar do dataset ter mais de 20k pares, não é de todo o suficiente para garantir boas decisões por parte da árvore. A melhor maneira de melhorar a árvore seria com um dataset ainda maior, uma vez que a qualidade dos dados em si serem bastante bons, visto serem fornecidos pelo MCTS.

####  3. Versatilidade

Conseguimos pensar e implementar muito mais melhorias tangíveis no MCTS do que na árvore para melhorar a performance do algoritmo. Isto para nós é um grande ponto a favor, pois permitiu mais creatividade e mais liberdade no desenvolvimento do algoritmo.
